In [1]:
from artifacts_ipv6_sra_scanning.config import *
import pickle
import os
import ipaddress
import glob
from tqdm.auto import tqdm
import country_converter as coco
from parallel_pandas import ParallelPandas
ParallelPandas.initialize(n_cpu=24, split_factor=1, disable_pr_bar=False)

2025-09-28 15:21:50.985 | INFO     | artifacts_ipv6_sra_scanning.config:<module>:22 - PROJ_ROOT path is: /home/service/artifacts-ipv6-sra-scanning


In [2]:
def load_and_join(files):
    df = pl.DataFrame()
    for i in tqdm(range(len(files))):
        if df.is_empty():
            df = pl.read_csv(files[i],columns=['saddr']).unique()
            df = df.with_columns(inscan=True)
        else:
            tmp = pl.read_csv(files[i],columns=['saddr']).unique()
            tmp = tmp.with_columns(inscan=True)
            df = df.join(tmp,on='saddr',how='full', suffix=f'_s{i}',coalesce=True)
    return df

In [3]:
file1 = '/home/service/ipv6-measurements-mkoch/backup_2024/sra-analysis/data/bgp48_sra_checks/zmap_icmp_bgp_48_sra_1728075185.log.zst'

In [4]:
file2 = '/home/service/ipv6-measurements-mkoch/backup_2024/sra-analysis/data/ripe90-rescan/zmap_icmp_bgp_48_sra_1740416512.log.zst'

In [5]:
files = [file1,file2]

In [6]:
df = load_and_join(files)

  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
df

saddr,inscan,inscan_s1
str,bool,bool
"""2a0a:5586:776:0:4eba:7dff:feb5…",null,true
"""240f:48:5a46:1:4eab:f8ff:fec8:…",true,true
"""2a0d:ce80:0:2:bee0:1ff:fe98:78…",null,true
"""2a02:a451:b158:0:ae64:62ff:fe6…",null,true
"""2001:4dd0:af0a:8b9d:e72:74ff:f…",null,true
…,…,…
"""2a00:f3b:ffa1:788f:fdc9:b805:b…",true,null
"""240e:368:140a:d8ef:ae6:3bff:fe…",true,null
"""fc00:1::24:4d01""",true,null


In [8]:
df.filter(pl.col('inscan') & pl.col('inscan_s1'))

saddr,inscan,inscan_s1
str,bool,bool
"""240f:48:5a46:1:4eab:f8ff:fec8:…",true,true
"""2a02:a446:e7be:0:8efd:deff:fe1…",true,true
"""2600:1700:41d0::1""",true,true
"""2a03:7846:d7c6:100:185:12:88:1…",true,true
"""2a02:a45e:e9db:0:7e16:89ff:fef…",true,true
…,…,…
"""2001:4dd0:af8f:4::1497""",true,true
"""240f:a9:4270:1:fe12:63ff:fe08:…",true,true
"""240f:7f:1a99:1:fe12:63ff:feb7:…",true,true


In [9]:
df.filter(pl.col('inscan'))

saddr,inscan,inscan_s1
str,bool,bool
"""240f:48:5a46:1:4eab:f8ff:fec8:…",true,true
"""2a02:a446:e7be:0:8efd:deff:fe1…",true,true
"""2600:1700:41d0::1""",true,true
"""2a03:7846:d7c6:100:185:12:88:1…",true,true
"""2a02:a45e:e9db:0:7e16:89ff:fef…",true,true
…,…,…
"""2a00:f3b:ffa1:788f:fdc9:b805:b…",true,null
"""240e:368:140a:d8ef:ae6:3bff:fe…",true,null
"""fc00:1::24:4d01""",true,null


In [10]:
len(df.filter(pl.col('inscan') & pl.col('inscan_s1')))/len(df.filter(pl.col('inscan'))) *100

56.47488816066456